# Movie Complexity via Symbolic Compression

This notebook implements an algorithm to estimate the **Kolmogorov Complexity** of a movie narrative.

## The Concept

We aim to compress a movie plot into a minimal set of symbolic operations. We define:
- **Axiomatic Symbols**: Fundamental atoms (Characters, Actions, Basic Objects).
- **Derived Symbols**: New concepts built from axioms (e.g., `REVENGE := KILL(ENEMY)`).
- **Program Length**: The total characters required to represent the plot + the definitions of all derived symbols.

We use an LLM (Llama 3.1) as the compression engine, iteratively prompting it to minimize the description length.

## The Subject

We will analyze **Star Wars: A New Hope**.

In [ ]:
import json
import re
import time
from typing import List, Dict, Tuple, Any
from ollama import chat, ChatResponse

class SymbolicCompressor:
    def __init__(self, model: str = 'llama3.1'):
        self.model = model
        
        # Fundamental symbols that don't need definition
        self.axioms = {
            "ROLES": ["HERO", "VILLAIN", "MENTOR", "PRINCESS", "COMPANION", "DROID", "PILOT", "LEADER"],
            "ACTIONS": ["FIGHT", "KILL", "RESCUE", "ESCAPE", "CAPTURE", "DESTROY", "LEARN", "USE", "GIVE", "GO_TO", "DIE", "BUY", "HIRE", "HIDE", "REVEAL"],
            "OBJECTS": ["WEAPON", "SHIP", "PLANET", "BASE", "PLANS", "FORCE", "MESSAGE"],
            "RELATIONS": ["FATHER", "SON", "MASTER", "FRIEND", "ENEMY"],
            "LOGIC": ["AND", "OR", "NOT", "CAUSES", "IF", "THEN", "="]
        }
        
        self.derived_symbols: Dict[str, str] = {}
        self.compressed_narrative: List[str] = []
        
    def calculate_complexity(self) -> int:
        """
        Calculate the current 'Kolmogorov Complexity' score.
        Score = Length of Compressed Narrative + Length of Symbol Definitions
        """
        narrative_len = sum(len(s) for s in self.compressed_narrative)
        # We count the length of the definition string
        dict_len = sum(len(k) + len(v) for k, v in self.derived_symbols.items())
        return narrative_len + dict_len

    def _build_prompt(self, fragment: str, current_derived: Dict[str, str]) -> str:
        return f"""
You are a Compression Engine. Your goal is to rewrite the NARRATIVE FRAGMENT using a symbolic language to minimize the total character count (Complexity Score).

### AXIOMS (Free to use):
{json.dumps(self.axioms, indent=2)}

### DERIVED SYMBOLS (Already defined, free to use):
{json.dumps(current_derived, indent=2)}

### NARRATIVE FRAGMENT:
"{fragment}"

### INSTRUCTIONS:
1. Represent the fragment as a string using available symbols.
2. You may define NEW derived symbols if it helps compress the text (e.g. if a complex pattern repeats).
   - Format for new symbols: SYMBOL_NAME := DEFINITION
   - A new symbol definition adds to the complexity cost, so only create one if the savings in the narrative outweigh the cost of the definition.
3. Output must be valid JSON.

### OUTPUT FORMAT:
{{
  "compressed_text": "...symbolic string...",
  "new_definitions": {{
     "NEW_SYMBOL": "...definition using axioms..."
  }},
  "reasoning": "...why this is minimal..."
}}
"""

    def compress_fragment(self, fragment: str, iterations: int = 3) -> str:
        """
        Compresses a single fragment, iterating to refine the result.
        """
        best_response = None
        best_score = float('inf')
        current_best_data = None
        
        print(f"Processing: '{fragment[:50]}...'")
        
        for i in range(iterations):
            try:
                prompt = self._build_prompt(fragment, self.derived_symbols)
                
                response = chat(model=self.model, messages=[{'role': 'user', 'content': prompt}])
                content = response.message.content
                
                # Helper to find JSON block
                match = re.search(r'\{.*\}', content, re.DOTALL)
                if not match:
                    print(f"  [Iter {i+1}] No JSON found.")
                    continue
                    
                data = json.loads(match.group(0))
                
                comp_text = data.get("compressed_text", "")
                new_defs = data.get("new_definitions", {})
                
                # Calculate local cost (just for this step decision)
                local_cost = len(comp_text) + sum(len(k)+len(v) for k,v in new_defs.items())
                
                print(f"  [Iter {i+1}] Cost: {local_cost} | Text: {comp_text}")
                
                if local_cost < best_score:
                    best_score = local_cost
                    current_best_data = data
                    
            except Exception as e:
                print(f"  [Iter {i+1}] Error: {e}")

        if current_best_data:
            # Commit the best result
            self.compressed_narrative.append(current_best_data["compressed_text"])
            self.derived_symbols.update(current_best_data.get("new_definitions", {}))
            return current_best_data["compressed_text"]
        else:
            print("  Failed to compress fragment.")
            return "[FAILED]"

In [ ]:
    def run(self, fragments: List[str]):
        print(f"Starting Compression on {len(fragments)} fragments...")
        start_time = time.time()
        
        for i, frag in enumerate(fragments):
            print(f"\n--- Fragment {i+1}/{len(fragments)} ---")
            result = self.compress_fragment(frag)
            current_total_complexity = self.calculate_complexity()
            print(f"Current Total Complexity Score: {current_total_complexity}")
            
        print(f"\nDone in {time.time() - start_time:.2f}s")

In [ ]:
# Star Wars: A New Hope - Narrative Fragments
star_wars_fragments = [
    "The Galactic Empire captures Princess Leia's ship. She hides the Death Star plans inside the droid R2-D2, who escapes to Tatooine with C-3PO.",
    "Luke Skywalker buys the droids. R2-D2 reveals Leia's message for Obi-Wan Kenobi. Luke finds Obi-Wan, who tells him about the Force.",
    "Luke and Obi-Wan hire Han Solo and Chewbacca to take them to Alderaan. They discover Alderaan is destroyed and are captured by the Death Star.",
    "They rescue Leia. Obi-Wan sacrifices himself fighting Darth Vader. The group escapes to the Rebel base.",
    "The Rebels analyze the plans. Luke joins the attack squadron. He uses the Force to destroy the Death Star."
]

# Initialize and Run
compressor = SymbolicCompressor(model='llama3.1')
compressor.run(star_wars_fragments)

In [ ]:
print("\n====== FINAL COMPRESSION RESULTS ======")
print(f"Final Complexity Score: {compressor.calculate_complexity()}")
print("\n--- Derived Symbol Dictionary ---")
for k, v in compressor.derived_symbols.items():
    print(f"{k} := {v}")

print("\n--- Compressed Narrative ---")
full_story = " -> ".join(compressor.compressed_narrative)
print(full_story)